In [2]:
from typing import Dict, Text

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_recommenders as tfrs

2022-12-02 15:39:57.485744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 15:39:57.741182: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 15:39:57.741223: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-02 15:39:58.315768: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [5]:
#carts_df = pd.read_csv('../cart_clustering/all_carts.csv')
carts_df = pd.read_csv('carts_df.csv')
all_libelles = np.unique(carts_df['LIBELLE'])
all_libelles[:5]

array(['2013 GD PARFUME F200 ML EVIDENCE',
       '25 LINGETTES DEFROISSANTES SV', 'AB SVA BRUM EXPR CORP FL150ML',
       "AB SVA JAMBES D'ETE  T75 INTER", 'AB SVA Soin Lift Vis T50 INTER'],
      dtype=object)

In [6]:
# creating the dataframe for unique liblles
libelles_df = pd.DataFrame(data = all_libelles,columns = ['item_libelle'])
libelles_df.to_csv('libelles_df.csv')

In [7]:
carts = tf.data.experimental.make_csv_dataset('carts_df.csv', batch_size=2)
items = tf.data.experimental.make_csv_dataset('libelles_df.csv', batch_size=2)

2022-12-02 15:40:35.292753: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-02 15:40:35.292809: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-02 15:40:35.292856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-IEIC9MD2): /proc/driver/nvidia/version does not exist
2022-12-02 15:40:35.294167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
for i in items.take(5):
    print(i)

OrderedDict([('', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([188, 175], dtype=int32)>), ('item_libelle', <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'LE PARFUM   EVIDENCE  VAPO 30ML', b'HYDRA LAIT DEMAQ FL200ML'],
      dtype=object)>)])
OrderedDict([('', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([194, 121], dtype=int32)>), ('item_libelle', <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'MASC VOLUME BRUN CN3 2009 5ML', b'FRAICHEUR SH DCHE CHEVREF 200'],
      dtype=object)>)])
OrderedDict([('', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 56, 199], dtype=int32)>), ('item_libelle', <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'CRAY KHOL NOIR 81 LUM4 1,3gr', b'MASCARA LONG NOIR 01 CN3 8ML'],
      dtype=object)>)])
OrderedDict([('', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([99, 64], dtype=int32)>), ('item_libelle', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'EDT PN2 PECHE 20ML', b'CUBE DE BAIN MURE PN2 20G'], dtype=object)>)])
O

In [9]:
for c in carts.take(5):
    print(c)

OrderedDict([('', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([226, 278], dtype=int32)>), ('Unnamed: 0.1', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([226, 278], dtype=int32)>), ('Unnamed: 0', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([226, 278], dtype=int32)>), ('TICKET_ID', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([35464394, 33483238], dtype=int32)>), ('LIBELLE', <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'VAO BRILLANCE BRUN PEAU CN3 5.5ml', b'UMAJ GD CERISIERS FL200ML'],
      dtype=object)>)])
OrderedDict([('', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([441,  84], dtype=int32)>), ('Unnamed: 0.1', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([441,  84], dtype=int32)>), ('Unnamed: 0', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([441,  84], dtype=int32)>), ('TICKET_ID', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([34311187, 33609672], dtype=int32)>), ('LIBELLE', <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'SVC BAUME GRA

In [10]:
# Select the basic features.
carts = carts.map(lambda x: {"LIBELLE": x["LIBELLE"], "TICKET_ID": x["TICKET_ID"]})
items = items.map(lambda x: x["item_libelle"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# Build vocabularies to converts carts (tickets) ids and libelles into integer indices for embedding layers

ticket_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
ticket_ids_vocabulary.adapt(carts.map(lambda x: x["TICKET_ID"]))

In [ ]:
libelles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
libelles_vocabulary.adapt(items)